In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Web scraping dados.justica.gov.pt

## loading package and retrieving data

In [70]:
import requests
import pickle
import pandas as pd
from bs4 import BeautifulSoup

<IPython.core.display.Javascript object>

## funtions for scraping/data retrieval

In [52]:
def extract_html(url):
    r = requests.get(url)
    print(f"request status code: {r.status_code}")
    soup = BeautifulSoup(r.text, "html.parser")
    return soup


# def extract_html_csv(soup):
#     csv = [a['href'] for a in soup.find_all('a', href=True) if '/dataset/' in a['href'] and a['href'][-3:] == 'csv']
#     csv = pd.read_csv(csv[0], encoding='latin-1')
#     return csv


def extract_csv_url(soup):
    csv = [
        a["href"]
        for a in soup.find_all("a", href=True)
        if "/dataset/" in a["href"] and a["href"][-3:] == "csv"
    ]
    return csv

<IPython.core.display.Javascript object>

## scraping urls

In [4]:
urls = []
for page in range(1, 5): # 4 pages
    urls.append(f"https://dados.justica.gov.pt/dataset?page={page}")

r_ls = []
soup_ls = []
data_name = []
data_name_url = []
data_descricao = []
data_name_list = []
df_ls = []
for url in urls:
    r_ls.append(requests.get(url))
for r in r_ls:
    soup_ls.append(BeautifulSoup(r.text, "html.parser"))
for soup in soup_ls:
    data_name_list.append(soup.find_all("h3", class_="dataset-heading"))
    for k in data_name_list:
        for data in k:
            if data.find("a"):
                data_name_url.append(data.find("a")["href"])
                data_name.append(data.find("a").text)
df = pd.DataFrame(
    {
        "tema": data_name,
        #      "descricao": data_descricao,
        "url": data_name_url,
    }
)

<IPython.core.display.Javascript object>

In [5]:
df["url"] = ["https://dados.justica.gov.pt" + k for k in df["url"]]

<IPython.core.display.Javascript object>

In [22]:
df["url"][-3:]

179    https://dados.justica.gov.pt/dataset/nacionali...
180    https://dados.justica.gov.pt/dataset/nacionali...
181         https://dados.justica.gov.pt/dataset/sheet47
Name: url, dtype: object

<IPython.core.display.Javascript object>

In [48]:
# filtering empty object
filter_empty = ["sheet" not in i for i in df["url"]]
df = df.loc[filter_empty, :]

<IPython.core.display.Javascript object>

In [ ]:
df["url"] = [extract_csv_url(extract_html(url)) for url in df["url"]]

In [61]:
# save csv with urls of the data (csv files)
df.to_csv("dados-justica-gov-pt_urls_csvs.csv", index=None, encoding="latin-1")

<IPython.core.display.Javascript object>

## scraping content of csv files and saving it

In [65]:
csv_ls = [pd.read_csv(url[0], encoding="latin-1", chunksize=1000) for url in df["url"]]

<IPython.core.display.Javascript object>

In [75]:
csv_ls = [pd.concat(chunk) for chunk in csv_ls]

<IPython.core.display.Javascript object>

In [89]:
csv_ls = list(zip(df["tema"], csv_ls))

<IPython.core.display.Javascript object>

In [90]:
with open("csv_ls.pickle", "wb") as f:
    pickle.dump(csv_ls, f)
# load pickle file
# pickle.load(open("csv_ls.pickle", "rb"))

<IPython.core.display.Javascript object>